# 01_data_wrangling (local VS Code)
Combine ROI CSV files from data/raw into one cleaned file under data/processed for downstream analysis.

## Instructions
- Place your ROI CSVs in data/raw.
- Configure the columns to extract (defaults to 3rd and 7th).
- Run the cells to write data/processed/combined_output.csv.

## Step 1: Load configuration and set paths
This step loads `config.yaml`, sets input/output directories, and reads wrangling parameters such as columns to extract and frame rate. It prepares the workspace for combining raw CSVs.

**Output:** Initialized paths and configuration variables used by downstream cells.

In [28]:
# Config and imports
from pathlib import Path
import numpy as np
import pandas as pd
import yaml

CONFIG_PATH = Path("config.yaml")
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

RAW_DIR = Path(config["paths"]["raw_dir"])
PROCESSED_DIR = Path(config["paths"]["processed_dir"])
COMBINED_DATA_DIR = Path(config["paths"]["combined_data_dir"])
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
COMBINED_DATA_DIR.mkdir(parents=True, exist_ok=True)

wrangling_cfg = config["wrangling"]
columns_to_extract = wrangling_cfg["columns_to_extract"]  # zero-based column indices to keep from each file
index_col_name = wrangling_cfg["index_col_name"]
output_filename = wrangling_cfg["output_filename"]

analysis_cfg = config.get("analysis", {})
time_col_name = analysis_cfg.get("time_col", "Time (s) Adjusted")
frame_rate_hz = analysis_cfg.get("frame_rate_hz", 1.0)
if not frame_rate_hz or frame_rate_hz <= 0:
    raise ValueError(f"frame_rate_hz must be > 0 (got {frame_rate_hz})")

## Step 2: Combine raw CSVs into per-genotype outputs
This step finds all raw CSVs under the raw data directory, assigns each file to a genotype using token matching, and extracts the configured columns (Mean1/Mean2). It writes one combined CSV per genotype and optionally a combined file for all genotypes.

**Output:** `combined_output_<genotype>.csv` files in `paths.combined_data_dir` (and the optional combined file named by `wrangling.output_filename`).

In [30]:
# Load CSVs and combine selected columns into per-genotype outputs
import re

csv_paths = sorted(RAW_DIR.rglob("*.csv"))
if not csv_paths:
    raise FileNotFoundError(f"No CSV files found in {RAW_DIR.resolve()} (including subdirectories)")

def sanitize_name(name: str) -> str:
    return name.lower().replace(" ", "_").replace("-", "_")

def tokenize(text: str) -> list[str]:
    return [t for t in re.split(r"[^a-z0-9]+", text.lower()) if t]

def tokens_match(genotype_tokens: list[str], name_tokens: list[str], all_genotype_tokens: list[list[str]]) -> bool:
    # require exact token matches for this genotype
    name_token_set = set(name_tokens)
    if not all(gt in name_token_set for gt in genotype_tokens):
        return False
    
    # exclude if name matches a MORE SPECIFIC genotype (one with more tokens)
    for other_tokens in all_genotype_tokens:
        if len(other_tokens) > len(genotype_tokens):
            # this is a more specific genotype - check if name matches it
            if all(ot in name_token_set for ot in other_tokens):
                # name belongs to the more specific genotype, exclude it
                return False
    
    return True

genotypes = analysis_cfg.get("genotypes", [])
if not genotypes:
    raise ValueError("No genotypes listed in config.yaml under analysis.genotypes")

genotype_aliases = analysis_cfg.get("genotype_aliases", {})
if "wt" not in genotype_aliases:
    genotype_aliases["wt"] = ["wild type", "wildtype"]

# Prepare all genotype token sets for exclusion checking
all_genotype_tokens = []
for g in genotypes:
    g_tokens = tokenize(g)
    if g_tokens:
        all_genotype_tokens.append(g_tokens)
    for alias in genotype_aliases.get(g, []):
        alias_tokens = tokenize(alias)
        if alias_tokens:
            all_genotype_tokens.append(alias_tokens)

for genotype in genotypes:
    genotype_tokens = tokenize(genotype)
    if not genotype_tokens:
        raise ValueError(f"Invalid genotype name: '{genotype}'")

    alias_token_sets = [tokenize(alias) for alias in genotype_aliases.get(genotype, [])]
    token_sets = [genotype_tokens] + [tokens for tokens in alias_token_sets if tokens]

    genotype_paths = []
    for p in csv_paths:
        name_tokens = tokenize(p.name) + tokenize(p.parent.name)
        if any(tokens_match(tokens, name_tokens, all_genotype_tokens) for tokens in token_sets):
            genotype_paths.append(p)

    if not genotype_paths:
        raise FileNotFoundError(f"No CSV files matched genotype '{genotype}' in {RAW_DIR.resolve()}")

    combined_parts = []
    index_series = None
    time_series = None

    for path in genotype_paths:
        df = pd.read_csv(path)
        if df.shape[1] <= max(columns_to_extract):
            raise ValueError(f"{path.name} does not have the requested columns {columns_to_extract} (only has {df.shape[1]} columns)")
        extracted = df.iloc[:, columns_to_extract].copy()
        extracted.columns = [f"{path.stem}_" + str(c) for c in extracted.columns]
        if index_series is None:
            index_series = df.iloc[:, 0].rename(index_col_name)
            time_series = pd.Series(np.arange(len(df)) / frame_rate_hz, name=time_col_name)
        combined_parts.append(extracted)
        print(f"Loaded {path.name} -> {len(extracted.columns)} columns")

    combined_df = pd.concat([index_series, time_series] + combined_parts, axis=1)
    genotype_tag = sanitize_name(genotype)
    out_path = COMBINED_DATA_DIR / f"combined_output_{genotype_tag}.csv"
    combined_df.to_csv(out_path, index=False)
    print(f"Saved {out_path}")

# Also write a combined file for all genotypes (optional)
combined_parts = []
index_series = None
time_series = None
for path in csv_paths:
    df = pd.read_csv(path)
    if df.shape[1] <= max(columns_to_extract):
        raise ValueError(f"{path.name} does not have the requested columns {columns_to_extract} (only has {df.shape[1]} columns)")
    extracted = df.iloc[:, columns_to_extract].copy()
    extracted.columns = [f"{path.stem}_" + str(c) for c in extracted.columns]
    if index_series is None:
        index_series = df.iloc[:, 0].rename(index_col_name)
        time_series = pd.Series(np.arange(len(df)) / frame_rate_hz, name=time_col_name)
    combined_parts.append(extracted)

combined_df = pd.concat([index_series, time_series] + combined_parts, axis=1)
out_path = COMBINED_DATA_DIR / output_filename
combined_df.to_csv(out_path, index=False)
print(f"Saved {out_path}")
combined_df.head()

Loaded grouped_wild type 2M Sbfr_Series001_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild type 2M Sbfr_Series002_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild type 2M Sbfr_Series005_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild type 2M Sbfr_Series007_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild type 2M Sbfr_Series008_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild type 2M Sbfr_Series009_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild type 2M Sbfr_Series011_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild type 2M Sbfr_Series013_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild type 2M Sbfr_Series015_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild type 2M Sbfr_Series017_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild type 2M Sbfr_Series019_Crop001_GCamP_Results_260123.csv -> 2 columns
Loaded grouped_wild t

,Index,Time (s) Adjusted,grouped_osm8dr206 2M S bfr_Series002_Crop001_GCamP_Results_260123_Mean1,grouped_osm8dr206 2M S bfr_Series002_Crop001_GCamP_Results_260123_Mean2,grouped_osm8dr206 2M S bfr_Series004_Crop001_GCamP_Results_260123_Mean1,grouped_osm8dr206 2M S bfr_Series004_Crop001_GCamP_Results_260123_Mean2,grouped_osm8dr206 2M S bfr_Series005_Crop001_GCamP_Results_260123_Mean1,grouped_osm8dr206 2M S bfr_Series005_Crop001_GCamP_Results_260123_Mean2,grouped_osm8dr206 2M S bfr_Series007_Crop001_GCamP_Results_260123_Mean1,grouped_osm8dr206 2M S bfr_Series007_Crop001_GCamP_Results_260123_Mean2,...,grouped_wild type 2M Sbfr_Series019_Crop001_GCamP_Results_260123_Mean1,grouped_wild type 2M Sbfr_Series019_Crop001_GCamP_Results_260123_Mean2,grouped_wild type 2M Sbfr_Series021_Crop001_GCamP_Results_260123_Mean1,grouped_wild type 2M Sbfr_Series021_Crop001_GCamP_Results_260123_Mean2,grouped_wild type 2M Sbfr_Series023_Crop001_GCamP_Results_260123_Mean1,grouped_wild type 2M Sbfr_Series023_Crop001_GCamP_Results_260123_Mean2,grouped_wild type 2M Sbfr_Series024_Crop001_GCamP_Results_260123_Mean1,grouped_wild type 2M Sbfr_Series024_Crop001_GCamP_Results_260123_Mean2,grouped_wild type 2M Sbfr_Series025_Crop001_GCamP_Results_260123_Mean1,grouped_wild type 2M Sbfr_Series025_Crop001_GCamP_Results_260123_Mean2
0,1,0.0,7747.123,2574.508,12161.346,2462.802,14104.745,2615.235,15032.773,2728.679,...,6339.318,2405.409,8142.924,2576.946,12837.619,2269.699,35046.426,2887.387,19520.254,2516.253
1,2,0.1,7679.846,2543.400,12134.247,2447.000,13796.451,2577.333,14691.717,2691.962,...,6277.659,2374.875,7982.033,2554.000,12638.044,2261.513,34592.906,2841.267,19537.836,2486.101
2,3,0.2,7653.615,2540.877,12063.481,2431.519,13527.568,2568.627,14495.151,2662.094,...,6244.432,2351.943,7904.163,2555.207,12985.982,2258.319,34587.945,2850.947,19570.000,2493.405
3,4,0.3,7630.677,2539.938,12044.469,2436.580,13328.667,2560.961,14256.434,2655.792,...,6313.955,2371.739,7850.533,2552.641,13121.168,2268.018,34420.000,2836.253,19667.988,2488.177
4,5,0.4,7622.938,2532.354,11986.580,2437.716,13131.353,2540.765,14191.868,2669.755,...,6263.034,2365.739,7798.196,2559.978,13025.434,2262.257,33976.586,2832.360,19742.215,2486.848


## Step 3: Quick output summary
This step prints the shape and a preview of the combined output to verify the merge worked as expected.

**Output:** A small summary of the combined DataFrame size and column list.

In [20]:
# Quick summary
print(f"Combined shape: {combined_df.shape}")
print(f"Columns: {list(combined_df.columns)[:10]}{' ...' if len(combined_df.columns) > 10 else ''}")

Combined shape: (600, 94)
Columns: ['Index', 'Time (s) Adjusted', 'grouped_osm8dr206 ptr23dr180 2M Sbfr_Series002_Crop001_GCamP_Results_260123_Mean1', 'grouped_osm8dr206 ptr23dr180 2M Sbfr_Series002_Crop001_GCamP_Results_260123_Mean2', 'grouped_osm8dr206 ptr23dr180 2M Sbfr_Series004_Crop001_GCamP_Results_260123_Mean1', 'grouped_osm8dr206 ptr23dr180 2M Sbfr_Series004_Crop001_GCamP_Results_260123_Mean2', 'grouped_osm8dr206 ptr23dr180 2M Sbfr_Series006_Crop001_GCamP_Results_260123_Mean1', 'grouped_osm8dr206 ptr23dr180 2M Sbfr_Series006_Crop001_GCamP_Results_260123_Mean2', 'grouped_osm8dr206 ptr23dr180 2M Sbfr_Series011_Crop001_GCamP_Results_260123_Mean1', 'grouped_osm8dr206 ptr23dr180 2M Sbfr_Series011_Crop001_GCamP_Results_260123_Mean2'] ...
